In [1]:
import dicom
import numpy as np
import os
import glob
import re

/usr/local/lib/python3.5/dist-packages/dicom/__init__.py:53: UserWarning: 
This code is using an older version of pydicom, which is no longer 
maintained as of Jan 2017.  You can access the new pydicom features and API 
by installing `pydicom` from PyPI.
See 'Transitioning to pydicom 1.x' section at pydicom.readthedocs.org 
for more information.

  warnings.warn(msg)


In [2]:
#ACDCimgpath='/opt/data/acdc/niftidata/'
SCDimgpathT = '/opt/data/sunnybrook/challenge_training/'
SCDimgpathV = '/opt/data/sunnybrook/challenge_validation/'
SCDimgpathO = '/opt/data/sunnybrook/challenge_online/'
SCDlblpath = '/opt/output/sunnybrook/norm/1/3/labels/'

In [32]:
results = []
for root, _, files in os.walk(SCDlblpath):
    rootnode = root.split("/")
    #label = rootnode[-2]
    filename = rootnode[-1]
    #patient_path = SCDlblpath + filename + '/'
    
    if os.path.isdir(SCDimgpathT + filename + '/') is True:
        origimgpath = SCDimgpathT + filename + '/'
        patient_path = SCDlblpath + filename + '/'
        results.append([filename, patientVolume(patient_path, origimgpath)])
    
    elif os.path.isdir(SCDimgpathV + filename + '/') is True:
        origimgpath = SCDimgpathV + filename + '/'
        patient_path = SCDlblpath + filename + '/'
        results.append([filename, patientVolume(patient_path, origimgpath)])
    
    elif os.path.isdir(SCDimgpathO + filename + '/') is True:
        origimgpath = SCDimgpathO + filename + '/'
        patient_path = SCDlblpath + filename + '/'
        results.append([filename, patientVolume(patient_path, origimgpath)])

    #print (rootnode, patient_path)
    
    

/opt/output/sunnybrook/norm/1/3/labels//
/opt/output/sunnybrook/norm/1/3/labels/SC-N-11/


KeyboardInterrupt: 

In [29]:
b=np.load(patient_path +'IM-0944-0149.dcm.label.npy')
b.shape

(320, 320)

In [17]:
patients = []
est_esv = []
est_edv = []
for i in range(len(results)):
    patients.append(results[i][0])
    est_esv.append(results[i][1][0])
    est_edv.append(results[i][1][1])

In [24]:
checkESVVolumes(patients, est_esv)

SC-HF-NI-11  :  34.481483438174536
SC-HF-NI-3  :  9.35279958420685
SC-N-5  :  0.5916958689903176
SC-HF-NI-7  :  5.476280856686117
SC-HF-NI-15  :  19.861270932122522
SC-HYP-10  :  10.234004860952481
SC-HF-NI-12  :  43.595613615546654
SC-HF-I-9  :  166.18242647832346
SC-HF-I-10  :  20.103721298755715
SC-HF-NI-33  :  26.76510026205959
SC-HF-NI-36  :  13.496257032888963
SC-N-7  :  2.4700876542602757
SC-N-9  :  3.499267754056035
SC-HYP-8  :  20.165456504683064
SC-HF-NI-14  :  20.79067251032309
SC-HF-I-40  :  12.508775370637608
Normal: TP:  6 L1 Error:  6.561051277306628
HYP: TP:  10 L1 Error:  30.399461365635545
HFI: TP:  9 L1 Error:  198.7949231477168
HFNI: TP:  4 L1 Error:  173.81947823200835
Incorrect Estimates:  ['SC-HF-NI-11', 'SC-HF-NI-3', 'SC-N-5', 'SC-HF-NI-7', 'SC-HF-NI-15', 'SC-HYP-10', 'SC-HF-NI-12', 'SC-HF-I-9', 'SC-HF-I-10', 'SC-HF-NI-33', 'SC-HF-NI-36', 'SC-N-7', 'SC-N-9', 'SC-HYP-8', 'SC-HF-NI-14', 'SC-HF-I-40']


In [28]:
checkEDVVolumes(patients, est_edv)

SC-N-6  :  0.4300767360414284
SC-HF-I-7  :  17.320616476797483
SC-N-40  :  12.400300408678675
SC-N-5  :  7.435219738730652
SC-HF-NI-15  :  41.76700988623392
SC-HYP-10  :  35.437650519880776
SC-HF-NI-12  :  59.96215216299214
SC-HF-I-9  :  147.22226286011136
SC-HF-I-10  :  18.339802708247987
SC-HF-NI-33  :  8.925048341445574
SC-HF-I-12  :  15.97527666882857
SC-HYP-8  :  43.13111323222719
SC-HYP-11  :  10.309444346470222
SC-HF-NI-14  :  68.09105763963815
SC-HF-I-40  :  42.8373822659218
Normal: TP:  6 L1 Error:  20.265596883450755
HYP: TP:  9 L1 Error:  88.8782080985782
HFI: TP:  7 L1 Error:  241.6953409799072
HFNI: TP:  8 L1 Error:  178.74526803030977
Incorrect Estimates:  ['SC-N-6', 'SC-HF-I-7', 'SC-N-40', 'SC-N-5', 'SC-HF-NI-15', 'SC-HYP-10', 'SC-HF-NI-12', 'SC-HF-I-9', 'SC-HF-I-10', 'SC-HF-NI-33', 'SC-HF-I-12', 'SC-HYP-8', 'SC-HYP-11', 'SC-HF-NI-14', 'SC-HF-I-40']


In [11]:
def checkESVVolumes(patient,esv):
    
    NESV = [43.10, 14.74]
    nMin_ESV = NESV[0] - NESV [1]
    nMax_ESV = NESV[0] + NESV [1]
    nTP = 0
    nEsvError = 0
    
    HYPESV = [43.11,24.50 ]
    hypMin_ESV = HYPESV[0] - HYPESV [1]
    hypMax_ESV = HYPESV[0] + HYPESV [1]
    hypTP = 0
    hypEsvError = 0
    
    HFIESV = [174.34,90.64]
    hfiMin_ESV = HFIESV[0] - HFIESV [1]
    hfiMax_ESV = HFIESV[0] + HFIESV [1]
    hfiTP = 0
    hfiEsvError = 0
    
    HFNIESV = [158.28, 56.34]
    hfniMin_ESV = HFNIESV[0] - HFNIESV [1]
    hfniMax_ESV = HFNIESV[0] + HFNIESV [1]
    hfniTP = 0
    hfniEsvError = 0
    
    incorrect = []
    
    for i in range(len(patient)):
        if ('-N-') in patient[i]:
            if (nMin_ESV < esv[i]) and (esv[i] < nMax_ESV):
                nTP = nTP + 1
            else:
                tmp = min(abs(esv[i]-nMin_ESV), abs(esv[i]-nMax_ESV))
                nEsvError = nEsvError + tmp
                incorrect.append(patient[i])
                print (patient[i], ' : ', tmp)

        if ('-HYP-') in patient[i]:
            if hypMin_ESV <= esv[i] and esv[i] <= hypMax_ESV:
                hypTP = hypTP + 1
            else:
                tmp = min(abs(esv[i]-hypMin_ESV), abs(esv[i]-hypMax_ESV))
                hypEsvError = hypEsvError + tmp
                incorrect.append(patient[i])
                print (patient[i], ' : ', tmp)
                
        if ('-HF-I-') in patient[i]:
            if hfiMin_ESV <= esv[i] and esv[i] <= hfiMax_ESV:
                hfiTP = hfiTP + 1
            else:
                tmp = min(abs(esv[i]-hfiMin_ESV), abs(esv[i]-hfiMax_ESV))
                hfiEsvError = hfiEsvError + tmp
                incorrect.append(patient[i])
                print (patient[i], ' : ', tmp)
                
        if ('-HF-NI-') in patient[i]:
            if hfniMin_ESV <= esv[i] and esv[i] <= hfniMax_ESV:
                hfniTP = hfniTP + 1
            else:
                tmp = min(abs(esv[i]-hfniMin_ESV), abs(esv[i]-hfniMax_ESV))
                hfniEsvError = hfniEsvError + tmp
                incorrect.append(patient[i])
                print (patient[i], ' : ', tmp)
                
    print ('Normal: TP: ', nTP, 'L1 Error: ', nEsvError)
    print ('HYP: TP: ', hypTP, 'L1 Error: ', hypEsvError)
    print ('HFI: TP: ', hfiTP, 'L1 Error: ', hfiEsvError)
    print ('HFNI: TP: ', hfniTP, 'L1 Error: ', hfniEsvError)
    print ('Incorrect Estimates: ', incorrect)

In [12]:
def checkEDVVolumes(patient,edv):
    
    NEDV = [115.69, 36.89]
    nMin_EDV = NEDV[0] - NEDV [1]
    nMax_EDV = NEDV[0] + NEDV [1]
    nTP = 0
    nEdvError = 0
    
    HYPEDV= [114.39, 50.46 ]
    hypMin_EDV = HYPEDV[0] - HYPEDV [1]
    hypMax_EDV = HYPEDV[0] + HYPEDV [1]
    hypTP = 0
    hypEdvError = 0
    
    HFIEDV = [244.92, 86.02]
    hfiMin_EDV = HFIEDV[0] - HFIEDV [1]
    hfiMax_EDV = HFIEDV[0] + HFIEDV [1]
    hfiTP = 0
    hfiEdvError = 0
    
    HFNIEDV = [244.92, 63.21]
    hfniMin_EDV = HFNIEDV[0] - HFNIEDV [1]
    hfniMax_EDV = HFNIEDV[0] + HFNIEDV [1]
    hfniTP = 0
    hfniEdvError = 0
    
    incorrect = []
    
    for i in range(len(patient)):
        if ('-N-') in patient[i]:
            if nMin_EDV <= edv[i] and edv[i] <= nMax_EDV:
                nTP = nTP + 1
            else:
                tmp = min(abs(edv[i]-nMin_EDV), abs(edv[i]-nMax_EDV))
                nEdvError = nEdvError + tmp
                incorrect.append(patient[i])
                print (patient[i], ' : ', tmp)
                
        if ('-HYP-') in patient[i]:
            if hypMin_EDV <= edv[i] and edv[i] <= hypMax_EDV:
                hypTP = hypTP + 1
            else:
                tmp = min(abs(edv[i]-hypMin_EDV), abs(edv[i]-hypMax_EDV))
                hypEdvError = hypEdvError + tmp
                incorrect.append(patient[i])
                print (patient[i], ' : ', tmp)
                
        if ('-HF-I-') in patient[i]:
            if hfiMin_EDV <= edv[i] and edv[i] <= hfiMax_EDV:
                hfiTP = hfiTP + 1
            else:
                tmp = min(abs(edv[i]-hfiMin_EDV), abs(edv[i]-hfiMax_EDV))
                hfiEdvError = hfiEdvError + tmp
                incorrect.append(patient[i])
                print (patient[i], ' : ', tmp)
                
        if ('-HF-NI-') in patient[i]:
            if hfniMin_EDV <= edv[i] and edv[i] <= hfniMax_EDV:
                hfniTP = hfniTP + 1
            else:
                tmp = min(abs(edv[i]-hfniMin_EDV), abs(edv[i]-hfniMax_EDV))
                hfniEdvError = hfniEdvError + tmp
                incorrect.append(patient[i])
                print (patient[i], ' : ', tmp)
                
    print ('Normal: TP: ', nTP, 'L1 Error: ', nEdvError)
    print ('HYP: TP: ', hypTP, 'L1 Error: ', hypEdvError)
    print ('HFI: TP: ', hfiTP, 'L1 Error: ', hfiEdvError)
    print ('HFNI: TP: ', hfniTP, 'L1 Error: ', hfniEdvError)
    print ('Incorrect Estimates: ', incorrect)

In [31]:
def patientVolume(patient_path, origimgp):
    for root,_, files in os.walk(patient_path):
        
            norm = None
            #norm = 1
            slicedict={}
            print (patient_path)
        
            for i in files:
                #print (i)
                filenode = i.strip('.dcm.label.npy')
                filenode = re.split('-',filenode)
                #print (filenode)
                patient = filenode[1]
                #slicenum = 
                #frame = int(filenode[2])
                lblpath = root +'/'+ i
            
                imgpath = getDICOM(origimgp, patient, filenode[2])

                img = dicom.read_file(imgpath)
                
                if norm is not None:
                    ps = img.PixelSpacing[0]
                else:
                    ps = 1
            
                thickness = img.SliceThickness
                gap = img.SpacingBetweenSlices
                sliceLocation = slicelocation(img)
            
                lbl = np.load(lblpath)
                
                frac = getfrac(lbl)
                #area = frac * (ps * len(lbl))
                area = getarea(lbl, frac, ps)
                #area = frac * ((sum(i == 1 for i in img.flatten())) * len(lbl))
                volume = thickness * area

                if sliceLocation in slicedict:
                    slicedict[sliceLocation].update({filenode[2]:
                                                {'fraction': frac,
                                                 'Area': area, 
                                                 'Volume': volume}})
                else:
                    slicedict[sliceLocation] = {filenode[2]:
                                        {'fraction': frac,
                                        'Area': area, 
                                        'Volume': volume}}
            
            ESF, ESV, EDF, EDV = getVolumes(slicedict)
            #ESF, ESV, EDF, EDV = getVolumes_OneFrame(slicedict)
            #ESF, ESV, EDF, EDV = getVolumes_frustum(slicedict)
            #ESF, ESV, EDF, EDV = getVolumes_combo(slicedict)

            return (ESV, EDV)

In [14]:
def getDICOM(path, patient, img):
    imgpath = path + 'IM-' +patient +'-'+ img + '.dcm'
    return imgpath

In [15]:
def getfrac(img):
    frac = (sum(i == 1 for i in img.flatten()))/float(len(img.flatten()))
    return frac

In [16]:
def getarea(img, frac, ps): 
    #area = frac * ((sum(i ==1 for i in img.flatten())) * len(img))
    #area = frac * (ps * len(img))
    area = frac * (ps*len(img))**2
    return area

In [17]:
def slicelocation(img):
    sl= np.dot(img.ImagePositionPatient,
                   np.cross(img.ImageOrientationPatient[:3],img.ImageOrientationPatient[3:]))
    return sl

In [18]:
def getVolumes(slicedict):
    ESF = []
    ESVlist = []
    EDF = []
    EDVlist = []
    ESArea = []
    EDArea = []
    SL = []
    moreFrame = []
    oneFrame = []
    #print (slicedict.keys())

    for key in sorted(slicedict):
        #print (key)
        frames = list(slicedict[key].keys())
        #print (frames)
        if len(frames) > 2:
            print ('A slice has more than two frames')
            moreFrame.append([key, frames])
        if len(frames) < 2:
            print ('A slice has one frame')
            oneFrame.append([key, frames])
            
        if len(frames) == 2:
            SL.append(key)
            for i in range(len(frames)-1):
            #print (slicedict[key][frames[i]]['fraction'], slicedict[key][frames[i+1]]['fraction'] )
                if slicedict[key][frames[i]]['fraction'] < slicedict[key][frames[i+1]]['fraction']:
                    ESF.append(frames[i])
                    EDF.append(frames[i+1])
                    ESVlist.append(slicedict[key][frames[i]]['Volume'])
                    EDVlist.append(slicedict[key][frames[i+1]]['Volume'])
                    ESArea.append(slicedict[key][frames[i]]['Area'])
                    EDArea.append(slicedict[key][frames[i+1]]['Area'])
                
                else:
                    ESF.append(frames[i+1])
                    ESVlist.append(slicedict[key][frames[i+1]]['Volume'])
                    ESArea.append(slicedict[key][frames[i+1]]['Area'])
                    EDF.append(frames [i])
                    EDVlist.append(slicedict[key][frames[i]]['Volume'])
                    EDArea.append(slicedict[key][frames[i]]['Area'])
    #print (ESF, EDF)

        #ESV = sum(ESVlist)
        #EDV = sum(EDVlist)
        #SL = list(slicedict.keys())
        #SL.sort()
    ESV = 0
    EDV = 0
    #print (len(SL), len(ESVlist), len(EDVlist)
    
    for i in range(len(SL)-1):
        #print (i)
        ESV = ESV + (ESArea[i] + ESArea[i+1]) * (abs(SL[i+1]-SL[i])/2)
        EDV = EDV + (EDArea[i] + EDArea[i+1]) * (abs(SL[i+1]-SL[i])/2)
        
        #ESV = ESV + (ESArea[i] + ESArea[i+1] + np.sqrt(ESArea[i]*ESArea[i+1])) * (abs(SL[i+1]-SL[i])/2)
        #EDV = EDV + (EDArea[i] + EDArea[i+1] + np.sqrt(ESArea[i]*ESArea[i+1])) * (abs(SL[i+1]-SL[i])/2)
        
    ESV = ESV / 1000
    EDV = EDV / 1000
    
    #print (oneFrame)
    #print (moreFrame)
    return ESF, ESV, EDF, EDV

In [19]:
def getVolumes_frustum(slicedict):
    ESF = []
    ESVlist = []
    EDF = []
    EDVlist = []
    ESArea = []
    EDArea = []
    SL = []
    moreFrame = []
    oneFrame = []
    #print (slicedict.keys())

    for key in sorted(slicedict):
        #print (key)
        frames = list(slicedict[key].keys())
        #print (frames)
        if len(frames) > 2:
            print ('A slice has more than two frames')
            moreFrame.append([key, frames])
        if len(frames) < 2:
            print ('A slice has one frame')
            oneFrame.append([key, frames])
            
        if len(frames) == 2:
            SL.append(key)
            for i in range(len(frames)-1):
            #print (slicedict[key][frames[i]]['fraction'], slicedict[key][frames[i+1]]['fraction'] )
                if slicedict[key][frames[i]]['fraction'] < slicedict[key][frames[i+1]]['fraction']:
                    ESF.append(frames[i])
                    EDF.append(frames[i+1])
                    ESVlist.append(slicedict[key][frames[i]]['Volume'])
                    EDVlist.append(slicedict[key][frames[i+1]]['Volume'])
                    ESArea.append(slicedict[key][frames[i]]['Area'])
                    EDArea.append(slicedict[key][frames[i+1]]['Area'])
                
                else:
                    ESF.append(frames[i+1])
                    ESVlist.append(slicedict[key][frames[i+1]]['Volume'])
                    ESArea.append(slicedict[key][frames[i+1]]['Area'])
                    EDF.append(frames [i])
                    EDVlist.append(slicedict[key][frames[i]]['Volume'])
                    EDArea.append(slicedict[key][frames[i]]['Area'])
    #print (ESF, EDF)

        #ESV = sum(ESVlist)
        #EDV = sum(EDVlist)
        #SL = list(slicedict.keys())
        #SL.sort()
    ESV = 0
    EDV = 0
    #print (len(SL), len(ESVlist), len(EDVlist)
    
    for i in range(len(SL)-1):
        ESV = ESV + ((ESArea[i] + ESArea[i+1] + np.sqrt(ESArea[i]*ESArea[i+1])) * (abs(SL[i+1]-SL[i])/2))
        EDV = EDV + ((EDArea[i] + EDArea[i+1] + np.sqrt(ESArea[i]*ESArea[i+1])) * (abs(SL[i+1]-SL[i])/2))
        
    ESV = ESV / 1000
    EDV = EDV / 1000
    
    #print (oneFrame)
    #print (moreFrame)
    return ESF, ESV, EDF, EDV

In [20]:
def getVolumes_combo(slicedict):
    ESF = []
    ESVlist = []
    EDF = []
    EDVlist = []
    ESArea = []
    EDArea = []
    SL = []
    moreFrame = []
    oneFrame = []
    #print (slicedict.keys())

    for key in sorted(slicedict):
        #print (key)
        frames = list(slicedict[key].keys())
        #print (frames)
        if len(frames) > 2:
            print ('A slice has more than two frames')
            moreFrame.append([key, frames])
        if len(frames) < 2:
            print ('A slice has one frame')
            oneFrame.append([key, frames])
            
        if len(frames) == 2:
            SL.append(key)
            for i in range(len(frames)-1):
            #print (slicedict[key][frames[i]]['fraction'], slicedict[key][frames[i+1]]['fraction'] )
                if slicedict[key][frames[i]]['fraction'] < slicedict[key][frames[i+1]]['fraction']:
                    ESF.append(frames[i])
                    EDF.append(frames[i+1])
                    ESVlist.append(slicedict[key][frames[i]]['Volume'])
                    EDVlist.append(slicedict[key][frames[i+1]]['Volume'])
                    ESArea.append(slicedict[key][frames[i]]['Area'])
                    EDArea.append(slicedict[key][frames[i+1]]['Area'])
                
                else:
                    ESF.append(frames[i+1])
                    ESVlist.append(slicedict[key][frames[i+1]]['Volume'])
                    ESArea.append(slicedict[key][frames[i+1]]['Area'])
                    EDF.append(frames [i])
                    EDVlist.append(slicedict[key][frames[i]]['Volume'])
                    EDArea.append(slicedict[key][frames[i]]['Area'])
    #print (ESF, EDF)

        #ESV = sum(ESVlist)
        #EDV = sum(EDVlist)
        #SL = list(slicedict.keys())
        #SL.sort()
    ESV = 0
    EDV = 0
    #print (len(SL), len(ESVlist), len(EDVlist)
    
    for i in range(len(SL)-1):
        ESV = ESV + (ESArea[i] + ESArea[i+1]) * (abs(SL[i+1]-SL[i])/2)
        EDV = EDV + ((EDArea[i] + EDArea[i+1] + np.sqrt(ESArea[i]*ESArea[i+1])) * (abs(SL[i+1]-SL[i])/2))
        
    ESV = ESV / 1000
    EDV = EDV / 1000
    
    #print (oneFrame)
    #print (moreFrame)
    return ESF, ESV, EDF, EDV

In [21]:
def getVolumes_OneFrame(slicedict):
    ESF = []
    ESVlist = []
    EDF = []
    EDVlist = []
    ESArea = []
    EDArea = []
    SL = []
    moreFrame = []
    oneFrame = []
    #print (slicedict.keys())

    for key in sorted(slicedict):
        #print (key)
        frames = list(slicedict[key].keys())
        #print (frames)
        if len(frames) > 2:
            print ('A slice has more than two frames')
            moreFrame.append([key, frames])
        if len(frames) < 2:
            print ('A slice has one frame')
            oneFrame.append([key, frames])
            
        if len(frames) == 2:
            SL.append(key)
            for i in range(len(frames)-1):
            #print (slicedict[key][frames[i]]['fraction'], slicedict[key][frames[i+1]]['fraction'] )
                if slicedict[key][frames[i]]['fraction'] < slicedict[key][frames[i+1]]['fraction']:
                    ESF.append(frames[i])
                    EDF.append(frames[i+1])
                    ESVlist.append(slicedict[key][frames[i]]['Volume'])
                    EDVlist.append(slicedict[key][frames[i+1]]['Volume'])
                    ESArea.append(slicedict[key][frames[i]]['Area'])
                    EDArea.append(slicedict[key][frames[i+1]]['Area'])
                
                else:
                    ESF.append(frames[i+1])
                    ESVlist.append(slicedict[key][frames[i+1]]['Volume'])
                    ESArea.append(slicedict[key][frames[i+1]]['Area'])
                    EDF.append(frames [i])
                    EDVlist.append(slicedict[key][frames[i]]['Volume'])
                    EDArea.append(slicedict[key][frames[i]]['Area'])
    #print (ESF, EDF)

        #ESV = sum(ESVlist)
        #EDV = sum(EDVlist)
        #SL = list(slicedict.keys())
        #SL.sort()
    ESV = 0
    EDV = 0
    #print (len(SL), len(ESVlist), len(EDVlist)
    if len(oneFrame) > 0:
        SL_ESV = list(SL)
        SL_EDV = list(SL)
        #print (SL, SL_ESV, SL_EDV)
        for i in range(len(oneFrame)):
            potesv = 0
            potedv = 0
            area = slicedict[oneFrame[i][0]][oneFrame[i][1][0]]['Area']
            for j in range(len(ESArea)):
                potesv = potesv + abs(area - ESArea[j])
            for k in range(len(EDArea)):
                potedv = potedv + abs(area - EDArea[k])
            if potesv < potedv:
                ESArea.append(area)
                SL_ESV.append(oneFrame[i][0])
            else:
                EDArea.append(area)
                SL_EDV.append(oneFrame[i][0])
        
        #print (oneFrame[0][0],len(SL), len(SL_ESV), len(SL_EDV))
        for i in range(len(SL_ESV)-1):
            ESV = ESV + (ESArea[i] + ESArea[i+1]) * (abs(SL_ESV[i+1]-SL_ESV[i])/2)
        for i in range(len(SL_EDV)-1):
            EDV = EDV + (EDArea[i] + EDArea[i+1]) * (abs(SL_EDV[i+1]-SL_EDV[i])/2)
    
    else:
        for i in range(len(SL)-1):
        #print (i)
            ESV = ESV + (ESArea[i] + ESArea[i+1]) * (abs(SL[i+1]-SL[i])/2)
            EDV = EDV + (EDArea[i] + EDArea[i+1]) * (abs(SL[i+1]-SL[i])/2)
        
        #ESV = ESV + (ESArea[i] + ESArea[i+1] + np.sqrt(ESArea[i]*ESArea[i+1])) * (abs(SL[i+1]-SL[i])/2)
        #EDV = EDV + (EDArea[i] + EDArea[i+1] + np.sqrt(ESArea[i]*ESArea[i+1])) * (abs(SL[i+1]-SL[i])/2)
        
    ESV = ESV / 1000
    EDV = EDV / 1000
    
    #print (oneFrame)
    #print (moreFrame)
    return ESF, ESV, EDF, EDV